In [1]:
import os
import time
import pandas as pd

import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

2022-08-21 11:55:46.753775: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-21 11:55:46.753834: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
path = '../../datasets/test_dataset_model_pipeline/collected_statements_v0.5.csv'
df = pd.read_csv(path, sep='|', error_bad_lines=False)

## Topics

In [ ]:
#set topic label
candidate_label = ['Robot Design', 'Autonomous Robotics', 'Forex trading']

In [ ]:
#topic classifier
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [8]:
'''
for i in range(len(cl)):
    for k in cl[i]:
        print(cl[i][k])

sequences_list = []
topics_list = []
scores_topics_list = []
for i in range(len(cl)):
    sequences_list.append(cl[i]['sequence'])
    topics_list.append(cl[i]['labels'][0])
    scores_topics_list.append(cl[i]['scores'][0])
#print(sequences_list)
#print(topics_list)
#print(scores_topics_list)
df = pd.DataFrame(data={'sequence':sequences_list, 'labels':topics_list, 'scores':scores_topics_list})
'''

### Sample (only for exploration - delete in production)

In [ ]:
df = df.sample(n=200).reset_index()

### Run topic classification

In [ ]:
#topic classifier - labeling
def topics_classification(row, t0):
    cl = classifier(row['statement'], candidate_label, multi_label=False)
    cl = cl['labels'][0]
    if int(row.name) % 10 == 0:
         print(row.name, '/', df.shape[0], 'rows labeled in', int(round((time.time()-t0), 0)), 'seconds.')
    return cl

In [ ]:
t0 = time.time()
df['topic'] = df.apply(lambda row: topics_classification(row, t0), axis=1)

In [ ]:
# get distribution of topics
df.groupby('topic').size()

In [ ]:
# chop substring at the end of string
def rchop(s, suffix):
    if suffix and s.endswith(suffix):
        return s[:-len(suffix)]
    return s

In [ ]:
path = rchop(path, '.csv')
path_plus_topic = path + 'topic_labeled' + '.csv'

In [ ]:
df.to_csv(path_plus_topic, sep='|')